In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pickle

In [2]:
# Read in csv
df = pd.read_csv("LouisvilleCleanFinal.csv").drop(['Unnamed: 0'],axis=1)
df.head()

,PrimaryColor,BreedCategory,IntakeStatus,PetAgeCategory,Sex,TopBreed,BinaryOutcome
0,WHITE,PURE,NORMAL,YOUNG,MALE,PIT BULL TERRIER,DENY
1,BLACK,MIX,NORMAL,YOUNG,FEMALE,BORDER COLLIE,TAKE
2,TAN,MIX,NORMAL,YOUNG,MALE,GOLDEN RETRIEVER,TAKE
3,WHITE,PURE,NORMAL,BABY,FEMALE,LABRADOR RETRIEVER,TAKE
4,BLACK,PURE,SICK,YOUNG,MALE,PIT BULL TERRIER,DENY


In [3]:
# # use when different features need different preprocessing
# from sklearn.compose import make_column_transformer
# #  encoding of categorical features
# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder(sparse=False)

# column_trans = make_column_transformer(
#     (OneHotEncoder(), ['PrimaryColor', 'BreedCategory','IntakeStatus', 'PetAgeCategory','Sex', 'TopBreed']),
#     remainder='passthrough')

# y = ohe.fit_transform(df[['BinaryOutcome']])

# from sklearn.ensemble import RandomForestClassifier as rf

# clf = rf()
# clf.fit(X_train, y_train)


In [4]:
df.columns


Index(['PrimaryColor', 'BreedCategory', 'IntakeStatus', 'PetAgeCategory',
       'Sex', 'TopBreed', 'BinaryOutcome'],
      dtype='object')

In [19]:
X = df.drop(['PrimaryColor', 'BreedCategory', 'TopBreed', 'BinaryOutcome', 'IntakeStatus'], axis="columns")
y = df[['BinaryOutcome']]

# intakestatus, PetAgeCategory, Sex


In [20]:
# # example of fitting a model on the scaled dataset

# from sklearn.model_selection import train_test_split
# from sklearn.svm import SVC 
# from sklearn.ensemble import RandomForestClassifier
# from pickle import dump
# from sklearn.preprocessing import OneHotEncoder

# # define the encoder
# ohe = OneHotEncoder()

# # fit_transform the training dataset
# X = ohe.fit_transform(X).toarray()
# y = ohe.fit_transform(y).toarray()

# # define model and split data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# # # from sklearn.ensemble import RandomForestClassifier
# rf_model = RandomForestClassifier(n_estimators = 200, random_state = 20) 
# rf_model.fit(X_train, y_train)

In [21]:
# predict_y_test = rf_model.predict(X_test)
# from sklearn import metrics
# print("Accuracy score: ", metrics.accuracy_score(y_test, predict_y_test))

In [22]:
X.head(10)

,PetAgeCategory,Sex
0,YOUNG,MALE
1,YOUNG,FEMALE
2,YOUNG,MALE
3,BABY,FEMALE
4,YOUNG,MALE
5,BABY,FEMALE
6,SENIOR,FEMALE
7,YOUNG,MALE
8,YOUNG,MALE
9,ADULT,FEMALE


In [23]:
y.head()

,BinaryOutcome
0,DENY
1,TAKE
2,TAKE
3,TAKE
4,DENY


In [24]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

In [25]:
X = ohe.fit_transform(X).toarray()
# y = ohe.fit_transform(y).toarray()
print(X[0:10])

[[0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 0. 1.]
 [0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 1.]
 [0. 1. 0. 0. 1. 0.]
 [0. 0. 1. 0. 1. 0.]
 [0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 1. 0. 1.]
 [1. 0. 0. 0. 1. 0.]]


In [26]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier
from pickle import dump
from sklearn.preprocessing import OneHotEncoder
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [27]:
rf_model = RandomForestClassifier(n_estimators = 200, random_state = 20) 
rf_model.fit(X_train, y_train)



C:\Users\Lenovo\Anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=20, verbose=0,
                       warm_start=False)

In [28]:

# dump(rf_model, open('rf_modelTESTMarc.pkl', 'wb'))
# new_input = [[0, 1, 0, 0, 1, 0]]
# with open('rf_modelTESTMarc.pkl', 'rb') as file:  
#     model = pickle.load(file)
# model.predict(new_input) 

array(['DENY'], dtype=object)

In [ ]:
# pd.read_pickle('rf_modelTEST2.pkl')

In [41]:
import category_encoders as ce
te = ce.TargetEncoder()
te.fit_transform(X,y)

,0,1,2,3,4,5
0,0.0,0.0,0.0,1.0,0.0,1.0
1,0.0,0.0,0.0,1.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,1.0
3,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...
41530,1.0,0.0,0.0,0.0,0.0,1.0
41531,1.0,0.0,0.0,0.0,1.0,0.0
41532,1.0,0.0,0.0,0.0,0.0,1.0
41533,0.0,0.0,0.0,1.0,1.0,0.0
